In [1]:
# 1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.12，升级机器人电机驱动固件版本到0.2.10.11，详细使用请参考上位机使用说明
# 2. 请使用上位机对机器人零位进行标定（无绝编则不用）
# 3. 请使用上位机调整机器人GR1T2电机pd参数（注意不是pid参数），
#    参考值：上肢电机10-14/30-34电机 p：200，d：15;
#           上肢电机15-16/35-36电机 p：100，d:10;
#           腰部电机90              p:300，d:18;
#           腰部电机91-92            p:500, d:20。
#           头部电机93-95            p:70, d:10
#           腿部电机50-53/70-73电机 p:100,d:10;
#           脚踝电机54-55/74-75电机 p:100,d:10;
#    可以根据软硬自己调节,腕关节可以小一点
# 4. 请使用指定的docker镜像进行开发，镜像名为fourier_hardware:v2.4，请使用开源仓库中的docker启动脚本启动docker
# 5. 注意 t2 头部请在上电之前放在低头状态（pitch为0.8726）。roll 和 yaw 为0 。

In [2]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# grx_sot_py 用于对机器人进行运动学，动力学求解
import grx_sot_py
import numpy as np
from dataclasses import dataclass, field
import pickle
import time
import math

In [3]:
# 加载对应机型的config配置文件
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr1t2")

In [4]:
# 配置各控制组的控制模式为PD模式，并忽略下肢控制的运动
config.changeControlGroupMotorMode("left_manipulator","ignore")
config.changeControlGroupMotorMode("right_manipulator","ignore")
config.changeControlGroupMotorMode("left_wrist","ignore")
config.changeControlGroupMotorMode("right_wrist","ignore")
config.changeControlGroupMotorMode("head", "ignore")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_leg","pd")
config.changeControlGroupMotorMode("right_leg","pd")
config.changeControlGroupMotorMode("left_ankle","pd")
config.changeControlGroupMotorMode("right_ankle","pd")

True

In [5]:
# 计算各个输入的关节位置信息的误差
@dataclass
class PositionErrorMetrics:
    errors: np.ndarray = field(default_factory=lambda: np.array([]))
    rmse: float = 0.0
    max_error: float = 0.0
    mean_error: float = 0.0
    std_deviation: float = 0.0

    def calculate_metrics(self):
        if self.errors.size == 0:
            raise ValueError("Error array is empty, cannot calculate metrics.")

        # 均方根误差 (RMSE)
        self.rmse = np.sqrt(np.mean(np.square(self.errors)))

        # 最大误差 (Max Error)
        self.max_error = np.max(np.abs(self.errors))

        # 平均误差 (Mean Error)
        self.mean_error = np.mean(np.abs(self.errors))

    def display_metrics(self):
        print(f"RMSE: {self.rmse}")
        print(f"Max Error: {self.max_error}")
        print(f"Mean Error: {self.mean_error}")


In [6]:
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr1t2",config)
# 核心类，用于对gr1t2 绝编进行标定以及计算offset (gr1t2的绝编设置特殊，不同于gr2t2 和 grmini）
# 请程序每次关闭之后对机器人进行断电（！！！！！非常重要）
t2_abs_helper = fourier_hardware_py.T2ABSHelper("/usr/local/config/fourier_hardware/gr1t2")

In [7]:
# 根据配置文件定义对应的机器人类，支持运动学、动力学求解
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr1t2", grx_sot_py.Flags.ignore_collisions)

In [8]:
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joints_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joints_vel_order[group_name] = joint_vel_orders

In [9]:
print(config.getRobotName())
print(config.getControlGroupNames())
print(group_joints_pos_order)
print(group_joints_vel_order)
# 获取当前绝编位置，最多尝试3次，返回一个pair，代表成功与否，成功的话后面数组代表当前的绝编位置。
print(t2_abs_helper.getControlGroupABSAngle("waist",3))
print(t2_abs_helper.getControlGroupABSAngle("left_leg",3))
print(t2_abs_helper.getControlGroupABSAngle("right_leg",3))
print(t2_abs_helper.getControlGroupABSAngle("left_ankle",3))
print(t2_abs_helper.getControlGroupABSAngle("right_ankle",3))

gr1t2
['head', 'waist', 'left_manipulator', 'right_manipulator', 'left_wrist', 'right_wrist', 'left_leg', 'right_leg', 'left_ankle', 'right_ankle']
{'waist': [19, 20, 21], 'left_leg': [7, 8, 9, 10], 'right_leg': [13, 14, 15, 16], 'left_ankle': [11, 12], 'right_ankle': [17, 18]}
{'waist': [18, 19, 20], 'left_leg': [6, 7, 8, 9], 'right_leg': [12, 13, 14, 15], 'left_ankle': [10, 11], 'right_ankle': [16, 17]}
unknown error happened . 
(True, array([1.69734979, 0.54648066, 4.0205636 ]))
(True, array([3.09020424, 1.32382548, 3.13890815, 1.64135945]))
(True, array([1.67740798, 6.10064173, 0.57332534, 4.8807435 ]))
(True, array([1.83885944, 1.33072829]))
(True, array([5.32559776, 3.68347144]))


In [10]:
# 使能整个机器人
hardware.enableRobot()

MOTOR: 192.168.137.74, ENABLE SUCCESS! 
MOTOR: 192.168.137.75, ENABLE SUCCESS! 
MOTOR: 192.168.137.70, ENABLE SUCCESS! 
MOTOR: 192.168.137.71, ENABLE SUCCESS! 
MOTOR: 192.168.137.72, ENABLE SUCCESS! 
MOTOR: 192.168.137.73, ENABLE SUCCESS! 
MOTOR: 192.168.137.54, ENABLE SUCCESS! 
MOTOR: 192.168.137.55, ENABLE SUCCESS! 
MOTOR: 192.168.137.50, ENABLE SUCCESS! 
MOTOR: 192.168.137.51, ENABLE SUCCESS! 
MOTOR: 192.168.137.52, ENABLE SUCCESS! 
MOTOR: 192.168.137.53, ENABLE SUCCESS! 
MOTOR: 192.168.137.90, ENABLE SUCCESS! 
MOTOR: 192.168.137.91, ENABLE SUCCESS! 
MOTOR: 192.168.137.92, ENABLE SUCCESS! 


True

In [11]:
# 请注意在电机上电时，请确保头部处于低头状态！！！若初始状态不对请重新标定头部电机.
print(hardware.getControlGroupState("head").q)

[0.     0.     0.     0.     0.     0.     0.     0.     0.8726 0.    ]


In [12]:
# 生成机器人头部关节从标定位置到urdf零位的轨迹
head_point_init = [0.0, 0.0, 0.0]
head_state = hardware.getControlGroupState("head")
head_traj = fourier_hardware_py.bridgeTrajectory(head_state.q[-3:], head_point_init)

In [13]:
# 驱动头部控制组关节回到urdf零位
for i in range(len(head_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("head",head_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# 获取机器人当前状态下的对应控制组的关节qpos(rad),
lm = hardware.getControlGroupState("left_manipulator")
rm = hardware.getControlGroupState("right_manipulator")
print("left_manipulator:", lm.q)
print("right_manipulator", rm.q)

In [15]:
# 生成机器人双臂关节从当前位置到零位的轨迹
point_init = [0.0, 0.0, 0.0, 0.0, 0.0]
rm_traj = fourier_hardware_py.bridgeTrajectory(rm.q[-5:],point_init)
lm_traj = fourier_hardware_py.bridgeTrajectory(lm.q[-5:],point_init)

In [16]:
# 执行回零轨迹，驱动双臂控制组关节回到零位
for i in range(len(rm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("right_manipulator",rm_traj[1][i],np.zeros(5),gravity_torque[group_joints_vel_order["right_manipulator"]])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [17]:
# 驱动左臂控制组关节回到零位
for i in range(len(lm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    # regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    # params = robot.get_inertia_params()
    # gravity_torque = regressor.dot(params)
    hardware.setControlGroupPosCmd("left_manipulator",lm_traj[1][i],np.zeros(5),gravity_torque[group_joints_vel_order["left_manipulator"]])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [11]:
# 生成机器人腰部关节从当前位置到零位的轨迹
waist_point_init = [0.0, 0.0, 0.0]
waist_state = hardware.getControlGroupState("waist")
waist_traj = fourier_hardware_py.bridgeTrajectory(waist_state.q[-3:], waist_point_init)

In [12]:
# 驱动腰部控制组关节回到零位
for i in range(len(waist_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("waist",waist_traj[1][i],np.zeros(3),gravity_torque[group_joints_vel_order["waist"]])
    # hardware.setControlGroupPosCmd("waist",waist_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [36]:
# 生成机器人左腿关节从当前位置到零位的轨迹
left_leg_point_init = [0.0, 0.0, 0.0, 0.0]
left_leg_state = hardware.getControlGroupState("left_leg")
left_leg_traj = fourier_hardware_py.bridgeTrajectory(left_leg_state.q[-4:], left_leg_point_init)

left_ankle_point_init = [0.0, 0.0]
left_ankle_state = hardware.getControlGroupState("left_ankle")
left_ankle_traj = fourier_hardware_py.bridgeTrajectory(left_ankle_state.q[-2:], left_ankle_point_init)
left_leg_traj

(True,
 [array([ 0.09366351,  0.13094933, -0.26345868,  0.28540272]),
  array([ 0.09366265,  0.13094812, -0.26345626,  0.28540009]),
  array([ 0.09366007,  0.13094451, -0.263449  ,  0.28539223]),
  array([ 0.09365577,  0.13093851, -0.26343692,  0.28537915]),
  array([ 0.09364977,  0.13093012, -0.26342005,  0.28536087]),
  array([ 0.09364207,  0.13091936, -0.26339839,  0.2853374 ]),
  array([ 0.09363268,  0.13090622, -0.26337196,  0.28530878]),
  array([ 0.0936216 ,  0.13089073, -0.26334079,  0.285275  ]),
  array([ 0.09360883,  0.13087288, -0.26330488,  0.2852361 ]),
  array([ 0.09359439,  0.13085269, -0.26326425,  0.2851921 ]),
  array([ 0.09357827,  0.13083016, -0.26321893,  0.285143  ]),
  array([ 0.0935605 ,  0.13080531, -0.26316893,  0.28508883]),
  array([ 0.09354106,  0.13077814, -0.26311426,  0.28502961]),
  array([ 0.09351997,  0.13074865, -0.26305494,  0.28496535]),
  array([ 0.09349724,  0.13071687, -0.26299099,  0.28489608]),
  array([ 0.09347287,  0.13068279, -0.26292243, 

In [37]:
# 驱动左腿控制组+左腿脚踝控制组关节回到零位
for i in range(max(len(left_leg_traj[1]),len(left_ankle_traj[1]))):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    if i < len(left_leg_traj[1]):
        hardware.setControlGroupPosCmd("left_leg",left_leg_traj[1][i])
    if i < len(left_ankle_traj[1]):
        hardware.setControlGroupPosCmd("left_ankle",left_ankle_traj[1][i])
    # hardware.setControlGroupPosCmd("waist",waist_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [32]:
# 生成机器人右腿关节从当前位置到零位的轨迹
# right_leg_point_init = [-math.pi/20, math.pi/20, -math.pi/10, math.pi/10]
right_leg_point_init = [0.0, 0.0, 0.0, 0.0]
right_leg_state = hardware.getControlGroupState("right_leg")
right_leg_traj = fourier_hardware_py.bridgeTrajectory(right_leg_state.q[-4:], right_leg_point_init)

right_ankle_point_init = [0.0, 0.0]
right_ankle_state = hardware.getControlGroupState("right_ankle")
right_ankle_traj = fourier_hardware_py.bridgeTrajectory(right_ankle_state.q[-2:], right_ankle_point_init)
right_leg_traj

(True,
 [array([-0.00968561, -0.00955896,  0.11857727, -0.0349996 ]),
  array([-0.0096851 , -0.00955845,  0.11857096, -0.03499774]),
  array([-0.00968356, -0.00955693,  0.11855211, -0.03499218]),
  array([-0.009681  , -0.00955441,  0.11852083, -0.03498294]),
  array([-0.00967744, -0.00955089,  0.11847721, -0.03497007]),
  array([-0.00967288, -0.00954639,  0.11842137, -0.03495358]),
  array([-0.00966733, -0.00954091,  0.11835341, -0.03493353]),
  array([-0.0096608 , -0.00953447,  0.11827345, -0.03490992]),
  array([-0.00965329, -0.00952706,  0.11818158, -0.03488281]),
  array([-0.00964482, -0.0095187 ,  0.11807791, -0.03485221]),
  array([-0.0096354 , -0.0095094 ,  0.11796255, -0.03481816]),
  array([-0.00962503, -0.00949917,  0.11783561, -0.03478069]),
  array([-0.00961373, -0.00948801,  0.1176972 , -0.03473984]),
  array([-0.00960149, -0.00947594,  0.11754741, -0.03469562]),
  array([-0.00958834, -0.00946296,  0.11738636, -0.03464809]),
  array([-0.00957427, -0.00944907,  0.11721416, 

In [33]:
# 驱动右腿控制组+左腿脚踝控制组关节回到零位
for i in range(max(len(right_leg_traj[1]),len(right_ankle_traj[1]))):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    if i < len(right_leg_traj[1]):
        hardware.setControlGroupPosCmd("right_leg",right_leg_traj[1][i])
    if i < len(right_ankle_traj[1]):
        hardware.setControlGroupPosCmd("right_ankle",right_ankle_traj[1][i])
    # hardware.setControlGroupPosCmd("waist",waist_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [26]:
# 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
print(gravity_torque)
# 提取机器人对应控制组的关节重力补偿力矩
print(gravity_torque[group_joints_vel_order["waist"]])

[-1.84605270e-02 -1.37650928e-01  5.34978663e+02 -2.34129752e-01
  1.54178748e+00 -2.75777042e-02  9.29342794e-01 -3.17145475e-04
 -5.81483747e-01 -4.30091401e-01 -3.86676018e-01  8.53664677e-04
 -9.36555192e-01 -1.37290841e-03 -4.46704300e-01 -3.80327858e-01
 -3.82488917e-01 -2.26171767e-03  2.36012450e-03 -4.13269622e-01
  7.14991022e-02 -3.21052327e-02 -4.16587131e-01  9.30788772e-05
 -5.85486495e-02  9.15801561e-01 -3.06834107e-05 -5.12650180e-03
 -9.70455157e-06  2.77992995e-03 -2.36981600e-06 -5.83842535e-02
 -9.17636886e-01  6.05032058e-05 -5.88479711e-03  2.20103533e-05
 -2.78233124e-03 -3.81609900e-06]
[ 0.00236012 -0.41326962  0.0714991 ]


In [27]:
#记录位置误差及位置跟踪信息
position_error_list = [[] for _ in range(32)]
position_desired_list = []
position_real_list = []

In [46]:
position_desired = np.zeros(32)  # deg
ERROR_CODE = 50 #关节组报错的错误码

# 设置循环频率的时间
sim_dt = 0.0025
sim_duration = 40.00
sim_steps = int(sim_duration / sim_dt)

for k in range(sim_steps):
    start_time = time.perf_counter()
    # current control
    position_desired[0] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # waist_yaw_joint
    position_desired[1] = 15 * math.sin(0.0025 * k - math.pi / 2) + 15 # waist_pitch_joint
    position_desired[2] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # waist_roll_joint

    position_desired[3] = -(15 * math.sin(0.0025 * k - math.pi / 2) + 15) # head_roll_joint
    position_desired[4] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # head_pitch_joint
    position_desired[5] = -(15 * math.sin(0.0025 * k - math.pi / 2) + 15) # head_yaw_joint
    
    position_desired[6] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_pitch_joint
    position_desired[7] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # left_shoulder_roll_joint
    position_desired[8] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_yaw_joint
    position_desired[9] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_elbow_pitch_joint
    position_desired[10] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_wrist_yaw_joint
    position_desired[11] = -(10 * math.sin(0.0025 * k)) # left_wrist_roll_joint
    position_desired[12] = -(10 * math.sin(0.0025 * k)) # left_wrist_pitch_joint
    
    position_desired[13] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_pitch_joint
    position_desired[14] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_roll_joint
    position_desired[15] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_shoulder_yaw_joint
    position_desired[16] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_elbow_pitch_joint
    position_desired[17] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_wrist_yaw_joint
    position_desired[18] = -(10 * math.sin(0.0025 * k)) # right_wrist_roll_joint
    position_desired[19] =  -(10 * math.sin(0.0025 * k)) # right_wrist_pitch_joint

    position_desired[20] = (5 * math.sin(0.0025 * k - math.pi / 2) + 5) # left_shoulder_pitch_joint
    position_desired[21] = (10 * math.sin(0.0025 * k - math.pi / 2) + 10) # left_shoulder_roll_joint
    position_desired[22] = -(10 * math.sin(0.0025 * k - math.pi / 2) + 10) # left_shoulder_yaw_joint
    position_desired[23] = (10 * math.sin(0.0025 * k - math.pi / 2) + 10) # left_elbow_pitch_joint
    position_desired[24] = -(5 * math.sin(0.0025 * k - math.pi / 2) + 5) # left_wrist_yaw_joint
    position_desired[25] = -(5 * math.sin(0.0025 * k)) # left_wrist_roll_joint

    position_desired[26] = -(5 * math.sin(0.0025 * k - math.pi / 2) + 5) # right_shoulder_pitch_joint
    position_desired[27] = -(5 * math.sin(0.0025 * k - math.pi / 2) + 5) # right_shoulder_roll_joint
    position_desired[28] = -(10 * math.sin(0.0025 * k - math.pi / 2) + 10) # right_shoulder_yaw_joint
    position_desired[29] = (10 * math.sin(0.0025 * k - math.pi / 2) + 10) # right_elbow_pitch_joint
    position_desired[30] = -(5 * math.sin(0.0025 * k - math.pi / 2) + 5) # right_wrist_yaw_joint
    position_desired[31] = -(5 * math.sin(0.0025 * k)) # right_wrist_roll_joint

    #将控制命令从关节角度转换为弧度
    position_desired = np.deg2rad(position_desired)

    # 返回一个字典，字典的key是control group 的名字，value是连续失去连接的次数（最大观测值是50），如果value等于50，代表该control group 失去连接
    # 应该在程序控制周期内实时观测次值，保证程序运行的安全
    motor_stats = hardware.getGroupsLoss()
    #检查机器人真机的硬件状态，如果机器人使能后某个电机断连，将会切断控制循环命令，并答应错误信息。
    if ERROR_CODE in motor_stats.values():
        print(f"motor get error.")
        break

    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)

    # 可以分别取消对应control group来进行分区控制，同时注意是否设置前馈力。
    # 针对腰部控制组进行控制
    # hardware.setControlGroupPosCmd("waist", position_desired[0:3], np.zeros(3), gravity_torque[group_joints_vel_order["waist"]])
    # 针对头部控制组进行控制
    # hardware.setControlGroupPosCmd("head", position_desired[3:6], np.zeros(3), gravity_torque[group_joints_vel_order["head"]])
    # 针对头部控制组进行控制，没有前馈项
    # hardware.setControlGroupPosCmd("head", position_desired[3:6])
    # 针对双臂控制组进行控制，需要注意对应的输入：关节控制组、对应维度的关节位置信息、速度信息(设置为0)、前馈的力矩值
    # hardware.setControlGroupPosCmd("left_manipulator", position_desired[6:11], np.zeros(5), gravity_torque[group_joints_vel_order["left_manipulator"]])
    # hardware.setControlGroupPosCmd("right_manipulator", position_desired[13:18], np.zeros(5), gravity_torque[group_joints_vel_order["right_manipulator"]])
    # 针对双臂控制组进行控制，没有前馈项
    hardware.setControlGroupPosCmd("left_manipulator", position_desired[6:11])
    hardware.setControlGroupPosCmd("right_manipulator", position_desired[13:18])
    # 针对双臂腕部控制组进行控制
    hardware.setControlGroupPosCmd("left_wrist", position_desired[11:13])
    hardware.setControlGroupPosCmd("right_wrist", position_desired[18:20])

    # 针对双腿控制组进行控制
    # hardware.setControlGroupPosCmd("left_leg", position_desired[20:24])
    # hardware.setControlGroupPosCmd("right_leg", position_desired[26:30])
    # 针对双腿脚踝控制组进行控制
    # hardware.setControlGroupPosCmd("left_ankle", position_desired[24:26])
    # hardware.setControlGroupPosCmd("right_ankle", position_desired[30:32])

    # 计算关节运动之间的误差，但需要计算的时候可以取消注释，需要对齐position_desired和position_real的维度
    # state = hardware.readRealRobotState()
    # position_desired_list.append(position_desired)
    # position_real = state.q[group_joints_pos_order["waist"] + group_joints_pos_order["head"] + 
    #                         group_joints_pos_order["left_manipulator"] + group_joints_pos_order["left_wrist"] + 
    #                         group_joints_pos_order["right_manipulator"] + group_joints_pos_order["right_wrist"] + 
    #                         group_joints_pos_order["left_leg"] + group_joints_pos_order["left_ankle"] + 
    #                         group_joints_pos_order["right_leg"] + group_joints_pos_order["right_ankle"]]
    # position_real_list.append(np.rad2deg(position_real))
    # position_error = []
    # # 计算机器人位置误差并记录
    # for i in range(32):
    #     position_error = position_desired[i] - position_real[i]
    #     position_error_list[i].append(position_error)
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)
    

In [30]:
# 计算位置跟踪误差并进行计算各关节跟踪效果
for i in range(len(position_error_list)):
    metrics = PositionErrorMetrics(errors=np.array(position_error_list[i]))
    metrics.calculate_metrics()
    # 打印输出结果
    metrics.display_metrics()

# 记录robot关节运动结果
data = {
        "position_desired_list": position_desired_list,
        "position_real_list": position_real_list,
}
file_path = "./move_test_gravity.pkl"
with open(file_path, "wb") as file:
    pickle.dump(data, file)
print(f"Data successfully written to {file_path}")


RMSE: 0.03230673033715377
Max Error: 0.04462800973512626
Mean Error: 0.029953397496251203
RMSE: 0.036698491924294946
Max Error: 0.06153378683132715
Mean Error: 0.03124386818963772
RMSE: 0.030546020575594992
Max Error: 0.04475141011905781
Mean Error: 0.02860839383909907
RMSE: 0.3179554776109258
Max Error: 0.5114218771777267
Mean Error: 0.2607246279229502
RMSE: 0.29969138539857093
Max Error: 0.4761645183892399
Mean Error: 0.24319310072635122
RMSE: 0.31087172880679254
Max Error: 0.4938193345814443
Mean Error: 0.25546346989520907
RMSE: 0.07314879391368041
Max Error: 0.10520457179466558
Mean Error: 0.06626968906042804
RMSE: 0.07479532179062374
Max Error: 0.10569659443267987
Mean Error: 0.06793023687075159
RMSE: 0.07112473829662172
Max Error: 0.10043086990222405
Mean Error: 0.06446410283051582
RMSE: 0.07074802242801456
Max Error: 0.10028181370919775
Mean Error: 0.06408193800954153
RMSE: 0.05120284174581273
Max Error: 0.07345707178161587
Mean Error: 0.04630280215930479
RMSE: 0.000309543017695

In [ ]:
# 去使能机器人电机
hardware.disableRobot()